# Shuffel nach Horizon

In [2]:
import json
import pandas as pd
pd.set_option('display.max_rows', 40)

import random

In [3]:
import sys, json
from pathlib import Path

# 1. Projekt-Root nur einmal bestimmen
PROJECT_ROOT = Path.cwd().parent.resolve()

# 2. sys.path fürs Importieren um PROJECT_ROOT erweitern
sys.path.insert(0, str(PROJECT_ROOT))

In [4]:
import utils.presenter as show
import utils.rolling_planning as rp

In [5]:
data_file = Path(PROJECT_ROOT, "data/jobshop_instances.json")

# Datei laden
with open(data_file, "r", encoding="utf-8") as f:
    jobshop_instances = json.load(f)

instance =  jobshop_instances["instance ft10"]

jssp_data, df_times = rp.init_jobs_with_arrivals(instance, 7, u_b_mmax = 0.8, 
                                                    generate_deadlines = True, deadlines_buffer_factor= 1.7)
show.print_jobs(jssp_data)

Job_000:  [[0, 29], [1, 78], [2, 9], [3, 36], [4, 49], [5, 11], [6, 62], [7, 56], [8, 44], [9, 21]]
Job_001:  [[0, 43], [2, 90], [4, 75], [9, 11], [3, 69], [1, 28], [6, 46], [5, 46], [7, 72], [8, 30]]
Job_002:  [[1, 91], [0, 85], [3, 39], [2, 74], [8, 90], [5, 10], [7, 12], [6, 89], [9, 45], [4, 33]]
Job_003:  [[1, 81], [2, 95], [0, 71], [4, 99], [6, 9], [8, 52], [7, 85], [3, 98], [9, 22], [5, 43]]
Job_004:  [[2, 14], [0, 6], [1, 22], [5, 61], [3, 26], [4, 69], [8, 21], [7, 49], [9, 72], [6, 53]]
Job_005:  [[2, 84], [1, 2], [5, 52], [3, 95], [8, 48], [9, 72], [0, 47], [6, 65], [4, 6], [7, 25]]
Job_006:  [[1, 46], [0, 37], [3, 61], [2, 13], [6, 32], [5, 21], [9, 32], [8, 89], [7, 30], [4, 55]]
Job_007:  [[2, 31], [0, 86], [1, 46], [5, 74], [4, 32], [6, 88], [8, 19], [9, 48], [7, 36], [3, 79]]
Job_008:  [[0, 76], [1, 69], [3, 76], [5, 51], [2, 85], [9, 11], [6, 40], [7, 89], [4, 26], [8, 74]]
Job_009:  [[1, 85], [0, 13], [2, 61], [6, 7], [8, 64], [9, 76], [5, 47], [3, 52], [4, 90], [7, 4

In [6]:
 df_times

,Job,Arrival,Deadline
0,Job_000,13.47,2570.261776
1,Job_001,109.02,3410.194192
2,Job_002,132.93,3809.531845
3,Job_003,180.34,4420.083325
4,Job_004,221.70,2765.545995
...,...,...,...
163,Job_163,9602.91,13091.798019
164,Job_164,9747.90,12304.691776
165,Job_165,9793.35,13094.524192
166,Job_166,9887.55,13564.151845


## Horizons Dataframe

In [8]:
import pandas as pd

def generate_planning_horizons(num_days, horizon_length):
    horizons = []
    minutes_per_day = 1440

    for start_day in range(0, num_days):
        end_day = start_day + horizon_length - 1
        start_time = start_day * minutes_per_day
        end_time = (end_day + 1) * minutes_per_day  # exklusiv

        horizon_id = f"{start_day:02d}-{end_day:02d}"

        horizons.append({
            'horizon': horizon_id,
            'day_start': start_day,
            'day_end': end_day,
            'start_time': start_time,
            'end_time': end_time
        })

    df = pd.DataFrame(horizons)
    return df

In [9]:
df_horizons = generate_planning_horizons(num_days=10, horizon_length=3)
df_horizons

,horizon,day_start,day_end,start_time,end_time
0,00-02,0,2,0,4320
1,01-03,1,3,1440,5760
2,02-04,2,4,2880,7200
3,03-05,3,5,4320,8640
4,04-06,4,6,5760,10080
5,05-07,5,7,7200,11520
6,06-08,6,8,8640,12960
7,07-09,7,9,10080,14400
8,08-10,8,10,11520,15840
9,09-11,9,11,12960,17280


## Zeiten um Horizon ergänzen

In [11]:
def assign_horizons(df_times, df_horizons):
    # Leere Spalte für Horizon anlegen
    df_times['arrival_horizon'] = None
    df_times['deadline_horizon'] = None

    for i, row in df_horizons.iterrows():
        mask = (df_times['Arrival'] >= row['start_time']) & (df_times['Arrival'] < row['end_time'])
        df_times.loc[mask, 'arrival_horizon'] = row['horizon']

        mask = (df_times['Deadline'] >= row['start_time']) & (df_times['Deadline'] < row['end_time'])

        # mask = (df_times['Deadline'] - Bearbeitungszeit >= row['start_time']) 
        # & (df_times['Deadline'] - Bearbeitungszeit < row['end_time']) df_times['Deadline']
        
        df_times.loc[mask, 'deadline_horizon'] = row['horizon']

    return df_times


In [12]:
df_times = assign_horizons(df_times, df_horizons)
df_times

,Job,Arrival,Deadline,arrival_horizon,deadline_horizon
0,Job_000,13.47,2570.261776,00-02,01-03
1,Job_001,109.02,3410.194192,00-02,02-04
2,Job_002,132.93,3809.531845,00-02,02-04
3,Job_003,180.34,4420.083325,00-02,03-05
4,Job_004,221.70,2765.545995,00-02,01-03
...,...,...,...,...,...
163,Job_163,9602.91,13091.798019,06-08,09-11
164,Job_164,9747.90,12304.691776,06-08,08-10
165,Job_165,9793.35,13094.524192,06-08,09-11
166,Job_166,9887.55,13564.151845,06-08,09-11


## Shuffle

In [14]:
def get_alternative_horizons(arrival_horizon, deadline_horizon):
    """
    Liefert eine Liste möglicher alternativer Horizonte, basierend auf:
    - der Länge des arrival_horizon
    - der Verschiebung zwischen arrival- und deadline_horizon

    Ziel ist es, einen alternativen Horizont zu finden, der
    in der Mitte zwischen beiden liegt (verschoben), aber die gleiche Länge hat.
    """
    a_start, a_end = map(int, arrival_horizon.split('-'))
    d_start, d_end = map(int, deadline_horizon.split('-'))

    shift = d_end - a_end         # horizontales Delta, z. B. 2
    length = a_end - a_start + 1  # z. B. 3 Tage lang

    alternatives = []

    # Erzeuge verschobene Alternativen zwischen a_start und d_start
    for s in range(a_start + 1, d_start + 1):
        e = s + length - 1
        h = f"{s:02d}-{e:02d}"
        alternatives.append(h)

    return alternatives

# Beispiel
get_alternative_horizons("00-02", "02-04")

['01-03', '02-04']

In [15]:
import random

def change_arrival_horizon_df(df, min_keep_ratio):
    """
    Für ausgewählte Zeilen wird ein alternativer Horizont generiert, 
    der zwischen arrival_horizon und deadline_horizon liegt.
    
    Mindestens `min_keep_ratio` der Zeilen innerhalb jeder arrival_horizon-Gruppe bleiben unverändert.
    Die geänderten Werte werden in der neuen Spalte 'new_arrival_horizon' gespeichert.
    """
    # Kopie des DataFrames, um Originaldaten zu erhalten
    df_new = df.copy()

    # Neue Spalte anlegen, zunächst identisch zu arrival_horizon
    df_new['new_arrival_horizon'] = df_new['arrival_horizon']

    # Nur solche Zeilen betrachten, bei denen arrival_horizon ≠ deadline_horizon
    mismatch_df = df_new[df_new['arrival_horizon'] != df_new['deadline_horizon']]

    # Gruppiere diese Zeilen nach arrival_horizon-Wert
    grouped = mismatch_df.groupby('arrival_horizon')

    for horizon, group in grouped:
        n = len(group)
        # Anzahl der Zeilen, die innerhalb dieser Gruppe verändert werden sollen
        k = round((1 - min_keep_ratio) * n)

        if k <= 0:
            # Wenn zu wenige Zeilen vorhanden sind oder min_keep_ratio sehr hoch ist, überspringen
            continue

        # Zufällige Auswahl von Zeilen (Indexwerte), die geändert werden sollen
        indices_to_change = random.sample(list(group.index), k)

        for idx in indices_to_change:
            # Werte
            row = df_new.loc[idx]
            arrival = row['arrival_horizon']
            deadline = row['deadline_horizon']

            # Generiere mögliche alternative Horizonte (ausgelagert in eigene Logik)
            options = get_alternative_horizons(arrival, deadline)

            if options:
                # Wähle zufällig einen der alternativen Horizonte aus
                df_new.at[idx, 'new_arrival_horizon'] = random.choice(options)
            else:
                # Falls keine Alternative erzeugt werden kann (z. B. unlogische Kombination)
                print(f"Keine gültige Alternative für {arrival} → {deadline} (Index {idx})!!!")
    return df_new



In [16]:
df_changed_times = change_arrival_horizon_df(df_times, min_keep_ratio=0.75)

column_order = ['Job', 'Arrival', 'Deadline', 'new_arrival_horizon', 'arrival_horizon', 'deadline_horizon']
df_changed_times = df_changed_times[column_order]
df_changed_times


,Job,Arrival,Deadline,new_arrival_horizon,arrival_horizon,deadline_horizon
0,Job_000,13.47,2570.261776,00-02,00-02,01-03
1,Job_001,109.02,3410.194192,00-02,00-02,02-04
2,Job_002,132.93,3809.531845,01-03,00-02,02-04
3,Job_003,180.34,4420.083325,00-02,00-02,03-05
4,Job_004,221.70,2765.545995,01-03,00-02,01-03
...,...,...,...,...,...,...
163,Job_163,9602.91,13091.798019,06-08,06-08,09-11
164,Job_164,9747.90,12304.691776,06-08,06-08,08-10
165,Job_165,9793.35,13094.524192,06-08,06-08,09-11
166,Job_166,9887.55,13564.151845,08-10,06-08,09-11


In [17]:
df_changed_times.head(23)

,Job,Arrival,Deadline,new_arrival_horizon,arrival_horizon,deadline_horizon
0,Job_000,13.47,2570.261776,00-02,00-02,01-03
1,Job_001,109.02,3410.194192,00-02,00-02,02-04
2,Job_002,132.93,3809.531845,01-03,00-02,02-04
3,Job_003,180.34,4420.083325,00-02,00-02,03-05
4,Job_004,221.70,2765.545995,01-03,00-02,01-03
5,Job_005,296.28,3506.833724,00-02,00-02,02-04
6,Job_006,365.29,3058.012478,00-02,00-02,02-04
7,Job_007,384.34,3873.228019,01-03,00-02,02-04
8,Job_008,400.84,4265.155672,00-02,00-02,02-04
9,Job_009,496.36,3991.720909,00-02,00-02,02-04
